In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBNirBioModule, RGBNirBioDataModule

dm = RGBNirBioDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
model = RGBNirBioModule.load_from_checkpoint('checkpoints/r18RGBNirBio-val_loss=6.35046-epoch=2.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff9878db9d0>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff9878db9d0>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdo

"backbone":         resnet18
"bio_dropout":      0.0
"bio_layers":       [256, 512]
"datamodule":       {'batch_size': 512, 'num_workers': 10, 'pin_memory': True}
"early_stopping":   True
"num_bio":          27
"optimizer":        Adam
"optimizer_params": {'lr': 0.001}
"pretrained":       True
"trainer":          {'gpus': 1, 'max_epochs': 30, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x7ff9713c0eb0>, 'enable_checkpointing': True, 'overfit_batches': 0, 'deterministic': True, 'precision': 16, 'callbacks': [<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff9713c0e80>, <pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x7ff9326e5550>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x7ff9326e5580>, <pytorch_lightning.callbacks.model_summary.ModelSummary object at 0x7ff9326e55b0>, <pytorch_lightning.callbacks.gradient_accumulation_scheduler.GradientAccumulationScheduler object at 0x7ff9326e55e0

In [6]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for batch in tqdm(dl):
    labels = batch['label']
    preds = model.predict(batch)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [01:15<00:00,  1.05it/s]


0.7182527909634319

In [7]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for batch in tqdm(dl):
    preds = model.predict(batch)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observation_ids = batch['observation_id']
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:24<00:00,  2.90it/s]


In [8]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
18820,20166878,224 246 3072 553 5045 2971 3034 5095 5152 4999...
34649,21879063,5322 6016 5064 9041 5311 6553 6263 7358 6368 5...
6372,10544660,348 815 47 591 10 603 769 262 576 213 450 1316...
21515,20459516,5045 5366 5273 389 5332 5095 2971 3072 224 494...
23998,20732590,5943 5146 3521 5109 6263 6833 5311 2822 3132 7...


In [9]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
9318,10443141,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2439,10115911,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
22631,20578366,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31454,21536522,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
35739,21995555,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
796,10037863,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
27740,21135885,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20288,20329097,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
29550,21331627,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18396,20117216,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
